In [35]:
from openprompt.data_utils import InputExample
import torch
import pandas as pd
import os
import json, csv
from abc import ABC, abstractmethod
from collections import defaultdict, Counter
from typing import List, Dict, Callable

from openprompt.utils.logging import logger

from openprompt.data_utils.utils import InputExample
from openprompt.data_utils.data_processor import DataProcessor

import pandas as pd
import numpy as np

from torchnlp.encoders import LabelEncoder

In [6]:
# raw icd 9 data
mimic_data_dir = "/home/niallt/NLP_DPhil/DPhil_projects/mimic-icd9-classification/clinical-longformer/data/intermediary-data/"
mimic_data = pd.read_csv(f"{mimic_data_dir}/notes2diagnosis-icd-train.csv_top_codes_filtered.csv")

In [21]:
mimic_data

,Unnamed: 0,text,label
0,1,: : : Sex: F Service: CARDIOTHORACIC Allergies...,4240
1,3,: : : Sex: F Service: NEONATOLOGY HISTORY: wee...,V3001
2,6,: : : Sex: M Service: CARDIOTHORACIC Allergies...,41041
3,7,: : : Sex: F Service: MEDICINE Allergies: Peni...,51881
4,8,: : : Sex: F Service: CARDIOTHORACIC Allergies...,3962
...,...,...,...
14355,28100,: : Service: HISTORY OF THE PRESENT ILLNESS: M...,56212
14356,28101,: : : Sex: F Service: SURGERY Allergies: Patie...,99662
14357,28102,: : Service: CARDIOTHORACIC Allergies: Penicil...,41071
14358,28103,: : : Sex: M Service: Neonatology HISTORY OF P...,V3001


In [44]:
class MimicProcessor(DataProcessor):
    # TODO Test needed
    def __init__(self):
        super().__init__()
#         self.labels = ["contradiction", "entailment", "neutral"]
        

    def get_examples(self, data_dir):
        path = data_dir
        examples = []
        df = pd.read_csv(path)
        self.label_encoder = LabelEncoder(np.unique(df.label).tolist(), reserved_labels = [])
        
        for idx, row in df.iterrows():
#             print(row)
            _, body, label = row
            label = self.label_encoder.encode(label)
#             print(f"body : {body}")
#             print(f"label: {label}")
#             print(f"labels original: {self.label_encoder.index_to_token[label]}")
            
            text_a = body.replace('\\', ' ')

            example = InputExample(
                guid=str(idx), text_a=text_a, label=int(label)-1)
            examples.append(example)
            break
                
        return examples

In [45]:
data = MimicProcessor().get_examples(data_dir = f"{mimic_data_dir}/notes2diagnosis-icd-train.csv_top_codes_filtered.csv")

In [46]:
data

[{
   "guid": "0",
   "label": 14,
   "meta": {},
   "text_a": ": : : Sex: F Service: CARDIOTHORACIC Allergies: Patient recorded as having No Known Allergies to Drugs : Chief Complaint: SOB with exertion, heart murmur since y/o Major Surgical or Invasive Procedure: Mitral valve replacement(mm CE tissue History of Present Illness: y/o female with known MVP who was diagnosed with a heart murmur at age . She was evaluated with serial TTE's which showed worsening MR. Echo showed LVEF % with Mitral valve regurgitant fraction of %. She denies any symptoms. Past Medical History: Hyperlipidemia, MVP/MR, Depression, Obesity Social History: social Etoh, live with mother, deniesDA or tobacco use Family History: noncontributory Physical Exam: y/o F in bed NAD Neuro AA&Ox, nonfocal Chest CTAB resp unlab median sternotomy stable, c/d/i no d/c, RRR no m/r/g chest tubes and epicardial wires removed. Abd S/NT/ND/BS+ EXT warm with trace edema Pertinent Results: RADIOLOGY Preliminary Report CHEST (PA & L